In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # Importing pyplot from matplotlib
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy.linalg import svd
import os
from scipy.io import loadmat
from sklearn import model_selection
from dtuimldmtools import (
    draw_neural_net,
    train_neural_net,
    visualize_decision_boundary,
    rlr_validate
)
import torch
plt.rcParams.update({'font.size': 12})

## Data preprocessing 

In [23]:
#filename = '../Data/SAheart.txt'
#df = pd.read_csv(filename)

url = "https://hastie.su.domains/ElemStatLearn/datasets/SAheart.data"
df = pd.read_csv(url)  


# Remove the first column "row.names"
df = df.drop(columns = "row.names", axis = 1)

# one hot encoding 
df['famhist'] = df['famhist']=='Present'

# Specify the columns you want to include in the boxplot
#columns_to_plot = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd', 'famhist']
columns_x = ['sbp', 'tobacco', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']
columns_y = ['ldl']

print(df[columns_x])
print(df[columns_y])
X = StandardScaler().fit_transform(df[columns_x])
y = StandardScaler().fit_transform(df[columns_y])




     sbp  tobacco  adiposity  typea  obesity  alcohol  age  chd
0    160    12.00      23.11     49    25.30    97.20   52    1
1    144     0.01      28.61     55    28.87     2.06   63    1
2    118     0.08      32.28     52    29.14     3.81   46    0
3    170     7.50      38.03     51    31.99    24.26   58    1
4    134    13.60      27.78     60    25.99    57.34   49    1
..   ...      ...        ...    ...      ...      ...  ...  ...
457  214     0.40      31.72     64    28.45     0.00   58    0
458  182     4.20      32.10     52    28.61    18.72   52    1
459  108     3.00      15.23     40    20.09    26.64   55    0
460  118     5.40      30.79     64    27.35    23.97   40    0
461  132     0.00      33.41     62    14.70     0.00   46    1

[462 rows x 8 columns]
       ldl
0     5.73
1     4.41
2     3.48
3     6.41
4     3.50
..     ...
457   5.98
458   4.41
459   1.59
460  11.61
461   4.82

[462 rows x 1 columns]


In [ ]:
N, M = X.shape
K = 10
CV = model_selection.KFold(K,shuffle=True)
# Define the model structure
# binary cross entropy loss
loss_fn = torch.nn.BCELoss()
max_iter = 10000
error_N = np.array([])
error_ann = []
error_baseline = []
lambdas = np.power(10.,range(-5,9)) #10^-1 til 10^3
#w_noreg = np.empty((M,K))
reg_error = np.empty((K,1))
lambda_low =  np.empty((K,1))


In [25]:
loss_fn = torch.nn.MSELoss()

for k, (train_index, test_index) in enumerate(CV.split(X,y)): 
    print('\nCrossvalidation fold: {0}/{1}'.format(k+1,K))    
    
    X_train = X[train_index,:]
    y_train = y[train_index]
    X_test = X[test_index,:]
    y_test = y[test_index] 


    opt_val_err, opt_lambda, mean_w_vs_lambda, train_err_vs_lambda, test_err_vs_lambda = rlr_validate(X_train, y_train, lambdas, 10)
    reg_error[k] = min(test_err_vs_lambda)
    lambda_low[k] = lambdas[np.argmin(test_err_vs_lambda)]

    #Xty = X_train.T @ y_train
    #XtX = X_train.T @ X_train
    #w_noreg[:,k] = np.linalg.solve(XtX,Xty).squeeze()
    #Error_test[k] = loss_fn(  torch.Tensor(y_test),  torch.Tensor(X_test @ w_noreg[:,k]))


    # Extract training and test set for current CV fold, 
    # and convert them to PyTorch tensors
    X_train = torch.Tensor(X[train_index,:] )
    y_train = torch.Tensor(y[train_index] )
    X_test = torch.Tensor(X[test_index,:] )
    y_test = torch.Tensor(y[test_index] )

    error_N = np.array([])
    for n_hidden in range(1,10):
        model = lambda: torch.nn.Sequential(
                            torch.nn.Linear(M, n_hidden), #M features to H hiden units
                            # 1st transfer function, either Tanh or ReLU:
                            torch.nn.Tanh(),                            #torch.nn.ReLU(),
                            torch.nn.Linear(n_hidden, 1), # H hidden units to 1 output neuron
                            )

        net, final_loss, learning_curve = train_neural_net(model,
                                                        loss_fn,
                                                        X=X_train,
                                                        y=y_train,
                                                        n_replicates=1,
                                                        max_iter=max_iter)
    
        y_test_est = net(X_test) # threshold output of sigmoidal function
        #y_test = y_test


        error_rate = loss_fn(y_test_est,y_test).item()
        error_N = np.append(error_N,error_rate) # store error rate for current CV fold 

    
    error_baseline.append(loss_fn(torch.mean(y_train),y_test).item())

    error_ann.append(error_N)

error_baseline = np.array(error_baseline)
reg_error = reg_error.squeeze()


Crossvalidation fold: 1/10

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.7765414	6.0637312e-06
		Final loss:
		1897	0.7744269	9.235928e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\dtuimldmtools\models\nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.7634174	3.7397047e-05
		2000	0.7335822	1.113133e-05
		Final loss:
		2821	0.7310763	9.783589e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.737012	0.00010835857
		2000	0.7045643	1.8357403e-05
		3000	0.696845	5.901881e-06
		4000	0.69370264	4.4679487e-06
		5000	0.68918234	8.129621e-06
		6000	0.6830763	9.860184e-06
		7000	0.67543554	1.0236458e-05
		8000	0.670044	6.582724e-06
		9000	0.6632367	7.1894956e-06
		10000	0.65932053	5.06255e-06
		Final loss:
		10000	0.65932053	5.06255e-06

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.7179044	6.6416294e-05
		2000	0.67867374	3.9344133e-05
		3000	0.6560336	2.7619488e-05
		4000	0.6391368	1.6506401e-05
		5000	0.6308447	9.637261e-06
		6000	0.62621295	5.901293e-06
		7000	0.6228915	4.9758683e-06
		8000	0.62018794	3.5559608e-06
		9000	0.6183488	2.5062182e-06
		10000	0.6170198	1.9320137e-06
		Final loss:
		10000	0.6170198	1.9320137e-06

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.72220534	0.00018863128
		2000	0.64782566	4.793347e-05


c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([47, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	0.8432254	0.000118739204
		2000	0.80009264	2.7041744e-05
		3000	0.78727037	3.6340914e-06
		4000	0.7856996	1.8965433e-06
		Final loss:
		4912	0.78456664	9.876276e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.7729901	1.9122715e-05
		2000	0.769241	3.4093289e-06
		3000	0.7631522	8.122669e-06
		4000	0.7580777	6.7617934e-06
		5000	0.7548992	1.737053e-06
		Final loss:
		5409	0.7544725	9.4802004e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.7788819	3.6501526e-05
		2000	0.7638066	1.4904713e-05
		3000	0.75032556	1.6920088e-05
		4000	0.7423964	5.8609035e-06
		5000	0.7327129	1.3096837e-05
		6000	0.7223348	2.128884e-05
		7000	0.71549016	6.747743e-06
		8000	0.71155727	4.523369e-06
		9000	0.70878625	3.3637473e-06
		10000	0.7068122	2.3612017e-06
		Final loss:
		10000	0.7068122	2.3612017e-06

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.74509996	3.775645e-05
		2000	0.7028735	2.0267118e-05
		3000	0.6946857	8.408416e-06
		4000	0.6844342	1.4281903e-05
		5000	0.6781785	7.997864e-

c:\Users\trine\anaconda3\envs\dtu02450\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([46, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	0.6746553	4.4790577e-05
		Final loss:
		1978	0.66985154	9.787997e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.71009004	0.00019252028
		2000	0.6592951	2.422839e-05
		3000	0.65173304	3.4753011e-06
		4000	0.650198	2.9334803e-06
		5000	0.6485474	2.2057113e-06
		6000	0.64742273	1.3809655e-06
		7000	0.6465104	1.4751083e-06
		8000	0.645633	1.2924742e-06
		9000	0.64407194	3.516639e-06
		10000	0.64200366	2.692399e-06
		Final loss:
		10000	0.64200366	2.692399e-06

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.65318584	7.117165e-05
		2000	0.6365738	1.685375e-05
		3000	0.6249294	2.8612641e-05
		4000	0.6049202	1.7834167e-05
		5000	0.5992644	3.978505e-06
		Final loss:
		5737	0.59826285	9.962943e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.6660664	9.395309e-05
		2000	0.62457246	4.237033e-05
		3000	0.60407585	2.3088432e-05
		4000	0.5950193	1.172006e-05
		5000	0.58893245	9.209838e-06
		6000	0.58347976	1.0930332e-05
		7000	0.5782084	7.628236e-06
		8000	0.5741055	7.05983e-06
	

In [27]:
error_ann_min = np.empty(K)
for fn, err in enumerate(error_ann):
    print(f"Fold: {fn+1}, Error: {round(min(err),4)}, hidden layers: {np.argmin(err)+1}, reg: {round(reg_error[fn],4)}, lambda: {lambda_low[fn,0]}  baseline: {round(error_baseline[fn],4)}")
    error_ann_min[fn] = min(err)
print(" ")
for fn, err in enumerate(error_ann):
    print(f"{fn+1} & {np.argmin(err)+1} & {round(min(err),4)} & {int(lambda_low[fn])} & {round(reg_error[fn],4)} &  \multicolumn{{2}}{{c}}{ {round(error_baseline[fn],4)} } \\\\")



Fold: 1, Error: 0.6228, hidden layers: 2, reg: 0.8402, lambda: 100.0  baseline: 0.8131
Fold: 2, Error: 0.5005, hidden layers: 2, reg: 0.83, lambda: 100.0  baseline: 0.8368
Fold: 3, Error: 0.4432, hidden layers: 1, reg: 0.8444, lambda: 100.0  baseline: 0.6141
Fold: 4, Error: 1.575, hidden layers: 8, reg: 0.7206, lambda: 100.0  baseline: 1.5982
Fold: 5, Error: 0.6278, hidden layers: 1, reg: 0.8163, lambda: 10.0  baseline: 0.8722
Fold: 6, Error: 0.9597, hidden layers: 1, reg: 0.7826, lambda: 10.0  baseline: 1.1513
Fold: 7, Error: 0.7252, hidden layers: 4, reg: 0.8191, lambda: 100.0  baseline: 0.8598
Fold: 8, Error: 0.7167, hidden layers: 1, reg: 0.8105, lambda: 10.0  baseline: 1.1115
Fold: 9, Error: 0.8749, hidden layers: 1, reg: 0.7905, lambda: 10.0  baseline: 1.3195
Fold: 10, Error: 0.9472, hidden layers: 4, reg: 0.7868, lambda: 10.0  baseline: 0.9269
 
1 & 2 & 0.6228 & 100 & 0.8402 &  \multicolumn{2}{c}{np.float64(0.8131)} \\
2 & 2 & 0.5005 & 100 & 0.83 &  \multicolumn{2}{c}{np.float64

C:\Users\trine\AppData\Local\Temp\ipykernel_26172\2076898574.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"{fn+1} & {np.argmin(err)+1} & {round(min(err),4)} & {int(lambda_low[fn])} & {round(reg_error[fn],4)} &  \multicolumn{{2}}{{c}}{ {round(error_baseline[fn],4)} } \\\\")


In [30]:
df_results = pd.DataFrame({
    'Fold': np.arange(1, K+1),
    'ANN Test Error': error_ann_min,
    'Best h': [np.argmin(err)+1 for err in error_ann],
    'RLR Test Error': reg_error,
    'Best lambda': lambda_low.flatten(),
    'Baseline Test Error': error_baseline
})

print(df_results)

   Fold  ANN Test Error  Best h  RLR Test Error  Best lambda  \
0     1        0.622766       2        0.840244        100.0   
1     2        0.500533       2        0.830008        100.0   
2     3        0.443192       1        0.844425        100.0   
3     4        1.575036       8        0.720578        100.0   
4     5        0.627850       1        0.816314         10.0   
5     6        0.959658       1        0.782591         10.0   
6     7        0.725247       4        0.819147        100.0   
7     8        0.716665       1        0.810483         10.0   
8     9        0.874900       1        0.790520         10.0   
9    10        0.947233       4        0.786772         10.0   

   Baseline Test Error  
0             0.813107  
1             0.836785  
2             0.614128  
3             1.598241  
4             0.872158  
5             1.151260  
6             0.859772  
7             1.111506  
8             1.319462  
9             0.926934  


In [28]:
conf_int = lambda z: st.t.interval(confidence=0.95, df=np.size(z)-1, loc=np.mean(z), scale=st.sem(z))

import scipy.stats as st
des = 10
print("95% Confidence Intervals:")
print(f"[{round(conf_int(error_ann_min-reg_error)[0],des)} , {round(conf_int(error_ann_min-reg_error)[1],des)}]")
print(f"[{round(conf_int(error_ann_min-error_baseline)[0],des)} , {round(conf_int(error_ann_min-error_baseline)[1],des)}]")
print(f"[{round(conf_int(reg_error-error_baseline)[0],des)} , {round(conf_int(reg_error-error_baseline)[1],des)}]")

p_value_error_ann_min_reg = st.ttest_1samp(error_ann_min-reg_error, 0)[1]
p_value_error_ann_min_baseline = st.ttest_1samp(error_ann_min-error_baseline, 0)[1]
p_value_reg_error_baseline = st.ttest_1samp(reg_error-error_baseline, 0)[1]

# Print the p-values
print(f"P-value for error_ann_min - reg_error: {p_value_error_ann_min_reg:.{des}f}")
print(f"P-value for error_ann_min - error_baseline: {p_value_error_ann_min_baseline:.{des}f}")
print(f"P-value for reg_error - error_baseline: {p_value_reg_error_baseline:.{des}f}")

print(st.ttest_1samp(error_ann_min-reg_error,0))



95% Confidence Intervals:
[-0.2620708427 , 0.2524701632]
[-0.3182431094 , -0.1038117178]
[-0.4361253542 , 0.0236712064]
P-value for error_ann_min - reg_error: 0.9672536960
P-value for error_ann_min - error_baseline: 0.0015944433
P-value for reg_error - error_baseline: 0.0730244222
TtestResult(statistic=np.float64(-0.0422089698425523), pvalue=np.float64(0.9672536960191944), df=np.int64(9))
